In [33]:
import os
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterio.features import geometry_mask
from shapely.geometry import mapping

In [34]:
# Load shapefile
shapefile_path = '/home/shuruyi/Documents/qgis/google_satelite/shapefile/pinus.shp'
shapes = gpd.read_file(shapefile_path)

# Load the image
image_path = '/home/shuruyi/Documents/qgis/google_satelite/google_satelite.tif'
image = rasterio.open(image_path)

# Output directory to save the extracted images
output_dir = '/home/shuruyi/Documents/qgis/google_satelite/extracted_images'

# Ensure the output directory exists or create it if not
os.makedirs(output_dir, exist_ok=True)

# Iterate through each shape and extract ROIs
for index, shape in shapes.iterrows():
    geometry = shape['geometry']
    label = int(shape['pinus'])  # Assuming 'pinus' field contains '1' or '0'

    # Check if the geometry is valid
    if geometry is not None:
        shape_geom = [mapping(geometry)]

        # Use rasterio to crop the image based on the shape's geometry
        out_image, out_transform = mask(image, shape_geom, crop=True, nodata=None, filled=False)

        # Ensure that out_image has four bands (RGBA)
        if out_image.shape[0] == 3:
            alpha_band = np.ones((1, out_image.shape[1], out_image.shape[2]), dtype=out_image.dtype) * 255
            out_image = np.concatenate((out_image, alpha_band), axis=0)

        # Generate a unique filename for each extracted ROI with label embedded
        output_filename = f"extracted_roi_{index}_label_{label}.tif"

        # Create the full output path
        output_path = os.path.join(output_dir, output_filename)

        # Update metadata to make sure the output image has the correct transparency
        meta = image.meta.copy()
        meta.update({'count': out_image.shape[0], 'dtype': 'uint8'})

        # Write the extracted ROI to a new file
        with rasterio.open(output_path, 'w', **meta) as dest:
            dest.write(out_image)
    else:
        print(f"Skipping shape at index {index} due to invalid geometry.")

Skipping shape at index 0 due to invalid geometry.
Skipping shape at index 1 due to invalid geometry.
Skipping shape at index 2 due to invalid geometry.
Skipping shape at index 3 due to invalid geometry.
Skipping shape at index 4 due to invalid geometry.
Skipping shape at index 5 due to invalid geometry.
Skipping shape at index 6 due to invalid geometry.


KeyboardInterrupt: 